# 第5章 入力処理：思考の連鎖推論

 - [一、環境設定](#一、環境設定)
 - [二、思考の連鎖プロンプト](#二、思考の連鎖プロンプト)
 - [三、内面独白（Inner monologue）](#三、内面独白（Inner-monologue）)

本章では、入力の処理、つまり一連のステップを通じて有用な出力を生成することに焦点を当てます。

時として、モデルは特定の質問に答える前に詳細な推論を行う必要があります。以前のコースを受講された方は、このような例を多く見てきたことでしょう。時として、モデルは急いで結論を出しすぎて推論プロセスでエラーを起こすことがあります。そのため、クエリを再構成して、モデルに最終的な答えを出す前に一連の関連する推論ステップを提供するよう求めることで、より長い時間をかけてより深く問題について考えることができます。

通常、モデルに問題を段階的に推論するよう求めるこの戦略を、思考の連鎖推論（chain of thought reasoning）と呼びます。

## 一、環境設定

前章と同様に、まずOpenAI APIを使用する環境を設定する必要があります

In [ ]:
import openai
# サードパーティライブラリのインポート

openai.api_key  = "sk-..."
# API_KEYの設定、ご自身のAPI_KEYに置き換えてください

# 以下は環境変数を使用した設定方法の例です。より安全です。参考までに、以降は触れません。
# import openai
# import os
# OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
# openai.api_key = OPENAI_API_KEY

In [ ]:
def get_completion_from_messages(messages, 
                                 model="gpt-3.5-turbo", 
                                 temperature=0, 
                                 max_tokens=500):
    '''
    OpenAI GPT3.5にアクセスする関数をカプセル化

    パラメータ: 
    messages: これはメッセージのリストで、各メッセージはrole（役割）とcontent（内容）を含む辞書です。役割は'system'、'user'、または'assistant'になります。内容は役割のメッセージです。
    model: 呼び出すモデル、デフォルトはgpt-3.5-turbo（ChatGPT）、内部テスト資格を持つユーザーはgpt-4を選択できます
    temperature: これはモデル出力のランダム性を決定します。デフォルトは0で、出力が非常に確定的になります。温度を上げると出力がよりランダムになります。
    max_tokens: これはモデル出力の最大トークン数を決定します。
    '''
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # モデル出力のランダム性を決定
        max_tokens=max_tokens, # モデル出力の最大トークン数を決定
    )
    return response.choices[0].message["content"]

## 二、思考の連鎖プロンプト

ここでは、モデルに結論を出す前に答えを段階的に推論するよう求めます。

In [3]:
delimiter = "####"
system_message = f"""
Follow these steps to answer the customer queries.
The customer query will be delimited with four hashtags,\
i.e. {delimiter}. 

Step 1:{delimiter} First decide whether the user is \
asking a question about a specific product or products. \
Product cateogry doesn't count. 

Step 2:{delimiter} If the user is asking about \
specific products, identify whether \
the products are in the following list.
All available products: 
1. Product: TechPro Ultrabook
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-UB100
   Warranty: 1 year
   Rating: 4.5
   Features: 13.3-inch display, 8GB RAM, 256GB SSD, Intel Core i5 processor
   Description: A sleek and lightweight ultrabook for everyday use.
   Price: $799.99

2. Product: BlueWave Gaming Laptop
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-GL200
   Warranty: 2 years
   Rating: 4.7
   Features: 15.6-inch display, 16GB RAM, 512GB SSD, NVIDIA GeForce RTX 3060
   Description: A high-performance gaming laptop for an immersive experience.
   Price: $1199.99

3. Product: PowerLite Convertible
   Category: Computers and Laptops
   Brand: PowerLite
   Model Number: PL-CV300
   Warranty: 1 year
   Rating: 4.3
   Features: 14-inch touchscreen, 8GB RAM, 256GB SSD, 360-degree hinge
   Description: A versatile convertible laptop with a responsive touchscreen.
   Price: $699.99

4. Product: TechPro Desktop
   Category: Computers and Laptops
   Brand: TechPro
   Model Number: TP-DT500
   Warranty: 1 year
   Rating: 4.4
   Features: Intel Core i7 processor, 16GB RAM, 1TB HDD, NVIDIA GeForce GTX 1660
   Description: A powerful desktop computer for work and play.
   Price: $999.99

5. Product: BlueWave Chromebook
   Category: Computers and Laptops
   Brand: BlueWave
   Model Number: BW-CB100
   Warranty: 1 year
   Rating: 4.1
   Features: 11.6-inch display, 4GB RAM, 32GB eMMC, Chrome OS
   Description: A compact and affordable Chromebook for everyday tasks.
   Price: $249.99

Step 3:{delimiter} If the message contains products \
in the list above, list any assumptions that the \
user is making in their \
message e.g. that Laptop X is bigger than \
Laptop Y, or that Laptop Z has a 2 year warranty.

Step 4:{delimiter}: If the user made any assumptions, \
figure out whether the assumption is true based on your \
product information. 

Step 5:{delimiter}: First, politely correct the \
customer's incorrect assumptions if applicable. \
Only mention or reference products in the list of \
5 available products, as these are the only 5 \
products that the store sells. \
Answer the customer in a friendly tone.

Use the following format:
Step 1:{delimiter} <step 1 reasoning>
Step 2:{delimiter} <step 2 reasoning>
Step 3:{delimiter} <step 3 reasoning>
Step 4:{delimiter} <step 4 reasoning>
Response to user:{delimiter} <response to customer>

Make sure to include {delimiter} to separate every step.
"""

In [ ]:
delimiter = "####"
system_message = f"""
お客様の問い合わせに答えるには、以下の手順に従ってください。お客様の問い合わせは4つのシャープ記号（#）で区切られます。つまり {delimiter} です。

ステップ 1:{delimiter} まず、ユーザーが特定の製品または製品について質問しているかどうかを判断します。製品カテゴリーは含まれません。

ステップ 2:{delimiter} ユーザーが特定の製品について質問している場合、その製品が以下のリストにあるかどうかを確認してください。利用可能なすべての製品：

製品：TechPro ウルトラブック
カテゴリー：コンピューターとノートパソコン
ブランド：TechPro
型番：TP-UB100
保証期間：1年
評価：4.5
特徴：13.3インチディスプレイ、8GB RAM、256GB SSD、Intel Core i5プロセッサー
説明：日常使いに適したスタイリッシュで軽量なウルトラブック。
価格：$799.99

製品：BlueWave ゲーミングノートパソコン
カテゴリー：コンピューターとノートパソコン
ブランド：BlueWave
型番：BW-GL200
保証期間：2年
評価：4.7
特徴：15.6インチディスプレイ、16GB RAM、512GB SSD、NVIDIA GeForce RTX 3060
説明：没入感のある体験を提供する高性能ゲーミングノートパソコン。
価格：$1199.99

製品：PowerLite コンバーチブル
カテゴリー：コンピューターとノートパソコン
ブランド：PowerLite
型番：PL-CV300
保証期間：1年
評価：4.3
特徴：14インチタッチスクリーン、8GB RAM、256GB SSD、360度ヒンジ
説明：レスポンシブなタッチスクリーンを備えた多用途コンバーチブルノートパソコン。
価格：$699.99

製品：TechPro デスクトップ
カテゴリー：コンピューターとノートパソコン
ブランド：TechPro
型番：TP-DT500
保証期間：1年
評価：4.4
特徴：Intel Core i7プロセッサー、16GB RAM、1TB HDD、NVIDIA GeForce GTX 1660
説明：仕事と遊びに適した強力なデスクトップコンピューター。
価格：$999.99

製品：BlueWave Chromebook
カテゴリー：コンピューターとノートパソコン
ブランド：BlueWave
型番：BW-CB100
保証期間：1年
評価：4.1
特徴：11.6インチディスプレイ、4GB RAM、32GB eMMC、Chrome OS
説明：日常的なタスクに適したコンパクトで手頃な価格のChromebook。
価格：$249.99

ステップ 3:{delimiter} メッセージに上記のリストの製品が含まれている場合、ユーザーがメッセージで行っている仮定をリストアップしてください。例えば、ノートパソコンXがノートパソコンYより大きい、またはノートパソコンZが2年保証があるなど。

ステップ 4:{delimiter} ユーザーが仮定を行った場合、製品情報に基づいてその仮定が正しいかどうかを判断してください。

ステップ 5:{delimiter} まず、該当する場合はお客様の誤った仮定を丁寧に訂正してください。利用可能な5つの製品リストにある製品のみを言及または参照してください。これらは店舗が販売する唯一の5つの製品だからです。友好的な口調でお客様に答えてください。

以下の形式を使用してください：
ステップ 1:{delimiter} <ステップ1の推論>
ステップ 2:{delimiter} <ステップ2の推論>
ステップ 3:{delimiter} <ステップ3の推論>
ステップ 4:{delimiter} <ステップ4の推論>
お客様への返信:{delimiter} <お客様への返信内容>

各ステップを区切るために必ず {delimiter} を含めてください。
"""

In [ ]:
user_message = f"""
by how much is the BlueWave Chromebook more expensive \
than the TechPro Desktop"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is asking a question about two specific products, the BlueWave Chromebook and the TechPro Desktop.
Step 2:#### The prices of the two products are as follows:
- BlueWave Chromebook: $249.99
- TechPro Desktop: $999.99
Step 3:#### The user is assuming that the BlueWave Chromebook is more expensive than the TechPro Desktop.
Step 4:#### The assumption is incorrect. The TechPro Desktop is actually more expensive than the BlueWave Chromebook.
Response to user:#### The BlueWave Chromebook is actually less expensive than the TechPro Desktop. The BlueWave Chromebook costs $249.99 while the TechPro Desktop costs $999.99.


In [ ]:
user_message = f"""BlueWave ChromebookはTechProデスクトップよりどれくらい高いですか？"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 

response = get_completion_from_messages(messages)
print(response)

In [ ]:
user_message = f"""
do you sell tvs"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

Step 1:#### The user is asking if the store sells TVs.
Step 2:#### The list of available products does not include any TVs.
Response to user:#### I'm sorry, but we do not sell TVs at this store. Our available products include computers and laptops.


In [ ]:
user_message = f"""テレビは売っていますか"""
messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

## 三、内面独白（Inner monologue）

特定のアプリケーションでは、モデルの推論プロセスをユーザーと共有することが適切でない場合があります。例えば、チュートリングアプリケーションでは、学生に自分で問題を解決するよう促したいかもしれませんが、学生の解決策に対するモデルの推論プロセスが答えを漏らしてしまう可能性があります。

内面独白は、この状況を緩和するために使用できる戦略であり、モデルの推論プロセスを隠す高度な方法です。

内面独白のアイデアは、モデルに答えを明かさない方法で部分的な出力を生成させることで、ユーザーが完全な推論プロセスを見ることができないようにすることです。目標は、これらの部分を構造化された形式で隠し、それらを渡すことを容易にすることです。その後、ユーザーに出力を提示する前に出力を変換し、部分的な出力のみが表示されるようにします。

In [ ]:
try:
    final_response = response.split(delimiter)[-1].strip()
except Exception as e:
    final_response = "申し訳ございません。現在問題が発生しています。別の質問をお試しください。"
    
print(final_response)

次の章では、複雑なタスクを処理するための新しい戦略を学びます。つまり、1つのプロンプトでタスク全体を完了しようとするのではなく、複雑なタスクを一連のより単純なサブタスクに分解することです。